## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [60]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AMD", "AAPL"],
    interval="1d",
    start_date="2021-01-01",
    end_date="2021-02-20",
)
toolbox

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toolbox(symbol=['AMD', 'AAPL'], interval=1d, start_date=2021-01-01, end_date=2021-02-20, provider=yfinance)

In [63]:
result = toolbox.technical.mandelbrot_channel(historical=False, window="1m")



In [64]:
result.to_polars()



date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2021-02-19,"""AAPL""",127.6294,127.643982,131.2318
2021-02-19,"""AMD""",75.7891,89.580002,107.3635


In [55]:
from humbldata.core.standard_models.toolbox.technical.mandelbrot_channel import (
    MandelbrotChannelData,
)
import datetime
import polars as pl

data = pl.LazyFrame(
    {
        "date": pl.date_range(
            start=datetime.date(2021, 1, 1),
            end=datetime.date(2021, 1, 3),
            eager=True,
        ),
        "symbol": ["AAPL", "AAPL", "AAPL"],
        "bottom_price": [100.0, 100.0, 100.0],
        "recent_price": [110.0, 110.0, 110.0],
        "top_price": [120.0, 120.0, 120.0],
    }
)
MandelbrotChannelData(data).collect()


SchemaError: column 'close_price' not in dataframe
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["date", "symbol", "bottom_price", "recent_price"]; PROJECT */5 COLUMNS; SELECTION: "None"